In [1]:
from dotenv import find_dotenv,load_dotenv
from transformers import pipeline

import warnings

import requests
import os
load_dotenv(find_dotenv())
warnings.filterwarnings('ignore')
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

/Users/supriyasindigerekumaraswmamy/Desktop/Thesis/Hugging-face-models/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-29 21:12:15.711814: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Image to Text


In [2]:
def img2text(url):
    pipe=pipeline("image-to-text", model="Salesforce/blip-image-captioning-large")
    text = pipe(url)[0]['generated_text']
    print(text)
    return text


In [4]:
import warnings
warnings.filterwarnings('ignore')
image=img2text('dataset.png')


a screenshot of a computer screen with a graph and a bar chart


In [ ]:
image

'a screenshot of a computer screen with a graph and a bar chart'

# Generate story using LLAMA3 for the image

In [5]:
import ollama

def generate_story(scenario):
    prompt = f"""You are a storyteller;
Generate a short story based on the {scenario} in less than 30 words;
Use the below output format;

context: {scenario}
Story:
"""
    response = ollama.chat(model='llama3', options={'temperature': 1}, messages=[{
        'role': 'user',
        'content': prompt,
    }])
    story = response['message']['content']
    print(story)
    return story


In [6]:
story = generate_story(image)

Here is a short story based on the screenshot:

**Context:** A screenshot of a computer screen with a graph and a bar chart

**Story:** The numbers told a tale of triumph. Dr. Patel's research on patient outcomes revealed a sharp spike in successful treatments after implementing her innovative therapy protocol. As she gazed at the bar chart, a smile spread across her face, knowing her hard work was paying off.


# Text to speech

In [ ]:
def text2speech(message,api_token):
    API_URL = "https://api-inference.huggingface.co/models/espnet/kan-bayashi_csmsc_conformer_fastspeech2"
    headers = {"Authorization": f"Bearer {api_token}"} 
    payloads = {
        'inputs': message
    }
 
    response = requests.post(API_URL, headers=headers, json=payloads)
    if response.status_code == 200:
        with open('audio.flac', 'wb') as file:
            file.write(response.content)
        print("Audio file created successfully.")
    else:
        print(f"Error: {response.status_code}")
        print(response.json())
   ### with open ('audio.mp3','wb') as file:
        #file.write(response.content)
    #return response.content
   

In [ ]:
story

'Here is a short story based on the given screenshot:\n\n**Context:**\n\n\nA simple graph displays a steady incline, as if it\'s tracking a journey. A bar chart alongside shows a series of milestones marked "Project X Completion", each one representing a significant step forward.\n\n**Story:**\n\nAs I gazed at the screen, memories flooded back to that fateful night when our team finally wrapped up Project X. The graph represented our collective effort – steady and consistent like our determination. And those bar chart markers? Each one told a story of triumph over obstacles, until the final hurdle was cleared. That moment, frozen in digital eternity, still gave me goosebumps.'

In [ ]:
import os
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")
text2speech(story,HUGGINGFACEHUB_API_TOKEN)

Audio file created successfully.


# UI

In [10]:
import streamlit as st
def main():
    st.set_page_config(page_title="img 2 story")
    st.header("Turn image into a story")
    uploaded_file=st.file_uploader("Choose an image...",type="jpg")

    if uploaded_file is not None:
        print(uploaded_file)
        bytes_data =uploaded_file.getvalue()
        with open(uploaded_file.name, 'wb')as file:
            file.write(bytes_data)
        st.image(uploaded_file,caption='Uploaded image.',use_column_width=True)
        scenario=img2text(uploaded_file.name)
        story =generate_story(scenario)
        text2speech(story)

        with st.expander("scenario"):
            st.write(scenario)
        with st.expander("story"):
            st.write(story)

        st.audio("audio.flac")





In [9]:
if __name__ =='__main__':
    main()